In [7]:
import networkx as nx
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Circle, HoverTool, TapTool, BoxSelectTool, OpenURL
from bokeh.models.graphs import from_networkx
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral8
from math import log
import csv
from csv import reader
import pickle
with open('tensorflow-networkx', 'rb') as f:
    gr = pickle.load(f)

new_nodes = list(gr.nodes())
new_weights = list(gr.nodes[name]['weight'] for name in gr.nodes)
new_edges = list(gr.edges())


In [8]:
G = nx.Graph()
G.add_nodes_from(new_nodes,color= 'blue')  
G.add_edges_from(new_edges)

#node_size = {k:5*v for k,v in G.degree()} 
#nx.set_node_attributes(G, node_size, 'node_size')
# create the plot
plot = figure(x_range=(-1, 1), y_range=(-1, 1))

plot.add_tools(HoverTool(tooltips=[("Concept", "@concept"), 
                                   ("degree", "@degree")]), 
               TapTool(), 
               BoxSelectTool())

graph = from_networkx(G, nx.spring_layout, iterations=15, scale=1, center=(0,0))

graph.node_renderer.data_source.data['concept'] = list(G.nodes())

graph.node_renderer.data_source.data['degree'] = list(v for k,v in G.degree())

graph.node_renderer.data_source.data['size'] = list(6*v for v in new_weights)

#graph.node_renderer.data_source.data['colors'] = Spectral8

#print(graph.node_renderer.data_source.data['degree'])
# set node size
from bokeh.transform import linear_cmap
graph.node_renderer.glyph = Circle(
    size='size', 
    fill_color=linear_cmap('degree', 'Spectral8', 0, 15)
)
url = "https://www.google.com/search?q=@concept&oq=@concept+&aqs=chrome..69i57j46j0l3j46j0.2127j0j1&sourceid=chrome&ie=UTF-8"
taptool = plot.select(type=TapTool)
taptool.callback = OpenURL(url=url)

plot.renderers.append(graph)
show(plot)

#important link: https://stackoverflow.com/questions/50420584/color-nodes-by-networkx-node-attribute-with-bokeh

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: degree, size [renderer: GlyphRenderer(id='9b4160f1-65d9-41c8-9a4c-bc647990c6ea', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: degree, size [renderer: GlyphRenderer(id='ebb0d6db-e8c8-4723-bcc4-d5a405777c51', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: degree, size [renderer: GlyphRenderer(id='4f3046b7-64dd-4047-9e24-c537a1e453d1', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: degree, size [renderer: GlyphRenderer(id='383b52f1-6218-4ed2-a26c-028115c158f4', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: degree, size [renderer: GlyphRenderer(id='1b3e5e11-c6f7-45bf-9b5e-5a5f4b77a8c3', ...)]
